In [47]:
import dataset
import pandas as pd
from xai_cola import data,ml_model,counterfactual_explainer,cola,counterfactual_refiner
from xai_cola.cola_explainer import COunterfactualwithLimitedActions
import joblib
import pickle

%reload_ext autoreload
%autoreload 2

In [48]:
dataset1 =dataset.GermanCreditDataset()
df1 = dataset1.get_dataframe()
df1.head(2)

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,1,2,1,0,1,1169,6,5,0
1,22,0,2,1,1,2,5951,48,5,1


In [49]:
x_without_target = df1.drop(columns=['Risk']).copy()
x_without_target.head(2)
columns_name = x_without_target.columns
columns_name

Index(['Age', 'Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account',
       'Credit amount', 'Duration', 'Purpose'],
      dtype='object')

#### 1.Configuration

In [50]:
scaled_lgbmcClassifier = joblib.load('scaled_LGBMClassifier.pkl')
print(f'---scaledLGBMClassifier.pkl has already been loaded---')

---scaledLGBMClassifier.pkl has already been loaded---


In [51]:
# 从文件加载Scaler
with open('scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)
print(f'--- scaler has already been loaded ---')

--- scaler has already been loaded ---


In [52]:
a = loaded_scaler.fit_transform(df1)
b = loaded_scaler.fit_transform(x_without_target)

In [53]:
scaled_data = loaded_scaler.fit_transform(x_without_target)

In [54]:
scaled_data

array([[ 2.76645648,  0.67028006,  0.14694918, ..., -0.74513141,
        -1.23647786,  1.07326286],
       [-1.19140394, -1.49191369,  0.14694918, ...,  0.94981679,
         2.24819436,  1.07326286],
       [ 1.18331231,  0.67028006, -1.38377145, ..., -0.41656241,
        -0.73866754,  0.06170503],
       ...,
       [ 0.21583532,  0.67028006,  0.14694918, ..., -0.87450324,
        -0.73866754,  1.07326286],
       [-1.10345149,  0.67028006,  0.14694918, ..., -0.50552769,
         1.9992892 ,  1.07326286],
       [-0.75164167,  0.67028006,  0.14694918, ...,  0.46245715,
         1.9992892 , -0.9498528 ]])

In [55]:
scaled_data.shape

(1000, 9)

In [56]:
data1 = data.NumpyData(scaled_data, target_name='Risk', feature_names=columns_name)   # numpydata has the get_x() method

In [57]:
scaled_ml_model = ml_model.Model(
    model = scaled_lgbmcClassifier,
    backend = 'sklearn'
    )

### use the alibi counterfactual explanier

In [70]:
data1.get_x()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,2.766456,0.670280,0.146949,-0.133710,-1.231393,-0.001045,-0.745131,-1.236478,1.073263
1,-1.191404,-1.491914,0.146949,-0.133710,-0.196609,1.044372,0.949817,2.248194,1.073263
2,1.183312,0.670280,-1.383771,-0.133710,-0.196609,-1.046463,-0.416562,-0.738668,0.061705
3,0.831502,0.670280,0.146949,-2.016956,-0.196609,-0.001045,1.634247,1.750384,0.567484
4,1.535122,0.670280,0.146949,-2.016956,-0.196609,-0.001045,0.566664,0.256953,-0.949853
...,...,...,...,...,...,...,...,...,...
995,-0.399832,-1.491914,-1.383771,-0.133710,-0.196609,-1.046463,-0.544162,-0.738668,0.567484
996,0.391740,0.670280,1.677670,-0.133710,-0.196609,-0.001045,0.207612,0.754763,-0.949853
997,0.215835,0.670280,0.146949,-0.133710,-0.196609,-1.046463,-0.874503,-0.738668,1.073263
998,-1.103451,0.670280,0.146949,-2.016956,-0.196609,-0.001045,-0.505528,1.999289,1.073263


In [71]:
from xai_cola.counterfactual_explainer import AlibiCounterfactualInstances 
alibi_test = AlibiCounterfactualInstances(ml_model=scaled_ml_model)

Predicted class for factual instance 0: [1]
Target class for counterfactual instance 0: 0
Predicted class for factual instance 1: [1]
Target class for counterfactual instance 1: 0
Predicted class for factual instance 2: [1]
Target class for counterfactual instance 2: 0
Predicted class for factual instance 3: [1]
Target class for counterfactual instance 3: 0


In [75]:
a,b = alibi_test.generate_counterfactuals(
    data=data1, 
    target_proba=0.51,
    tolerance=0.2,
    lam_init=1e-7,
    max_iter=8000,
    max_lam_steps=30
)
a_df = pd.DataFrame(a, columns=columns_name)
print(a_df)

        Age       Sex       Job   Housing  Saving accounts  Checking account  \
0 -0.663689 -1.491914 -1.383771 -0.133710        -0.196609          2.089789   
1 -0.839594  0.670280  0.146949 -0.133710        -0.196609          2.089789   
2  1.535122  0.670280  1.677670 -2.016956        -0.196609         -0.001045   
3  0.655598  0.670280  0.146949  1.749535        -0.196609         -0.001045   

   Credit amount  Duration   Purpose  
0      -0.895415 -0.987573  1.073263  
1       0.332731  1.252574  1.073263  
2       1.628221 -0.738668  0.567484  
3      -0.229062 -0.406794 -0.949853  


In [74]:
b = alibi_test.counterfactual
b_df = pd.DataFrame(b,columns=columns_name)
print(b_df)

        Age       Sex       Job   Housing  Saving accounts  Checking account  \
0 -0.663689 -1.491914 -1.383771 -0.133710        -0.196609          2.089789   
1 -0.839594  0.670280  0.146949 -0.133710        -0.196609          2.089789   
2  1.535122  0.670280  1.677670 -2.016956        -0.196609          0.000725   
3  0.655598  0.670280  0.146949  1.749535        -0.196609          0.049193   

   Credit amount  Duration   Purpose  
0      -0.886748 -0.987573  1.073263  
1       0.345637  1.252574  1.073263  
2       1.628221 -0.738668  0.567484  
3      -0.203897 -0.406794 -0.949853  
